<a href="https://colab.research.google.com/github/cagBRT/SentimentTextAnalysis/blob/master/Sentiment_Text_Analysis_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Clone the entire repo.
!git clone  https://github.com/cagBRT/SentimentTextAnalysis.git cloned-repo
%cd cloned-repo
!ls

In [0]:
from IPython.display import Image
def page(num):
    return Image("images/sentTextAna"+str(num)+ ".png" , width=600)

# **Pre-requisites:**<br>
Python - 2 day course is sufficient<br>
Keras understanding - intro course is sufficient<br>
logistic regression - understanding<br>
Deep neural networks - intro course is sufficient<br>
Hyperparameter tuning _ intro course is sufficient<br>



# **Pre-work**
**Do this step before coming to class. It may take as long as 30 minutes to complete the download and upload.**<br>
<br>
You will need to add a file to your google drive. <br>
1. Download the file to your computer. 
>Click on the link below. Then click **Download** <br>
The download can take as long as 15 minutes.<br>
The file to download is: [fileToAddToGoogleDrive](https://drive.google.com/open?id=1zJI1Xz-CgaQqX1UtBcOhUjEKWcSt6QK6)<br>
The file is large: 2GBytes<br><br>


2. Upload the file to Google Drive:<br>
>Open Google Drive<br>
On the Drive menu, click on **New** >> **File Upload**<br>
Find the file on your computer, click on it and upload the file. 

The file is large, it may take as long as 15 minutes<br>
Once the file is on you<br><br>
The file is from a website: [English word vectors](https://fasttext.cc/docs/en/english-vectors.html)<br>
This page gathers several pre-trained word vectors trained using fastText.

# **Sentiment Text Analyis**<br>
This notebook introduces basic machine learning methods for determing if yelp, Amazon, and imdb reviews are positive or negative. <br>
The following topics are covered in this notebook: 
1. Create and train a baseline model.
2. Create and train a simple DNN model
3. Add an embedded layer to a simple DNN model
4. Add a MaxPooling layer to an embedded model
5. Use a pretrained embedded model
6. Create a CNN model with an embedded layer
7. Do hyperparameter tuning using random search

It does not cover recurrent neural networks or LSTM or GRU. 

In [0]:
page(1)

**What is sentiment analysis?**<br>
Sentiment analysis is a branch of natural language processing. It is one of the most important sources in decision making for determining the sentiment of language. It can extract from, identify, evaluate, or characterize online sentiment of reviews.<br>

Sentiment analysis assumes various forms, from models that focus on polarity (positive, negative, neutral) to those that detect feelings and emotions (angry, happy, sad, etc), or even models that identify intentions (e.g. interested v. not interested).<br><br>

It’s estimated that 80% of the world’s data is unstructured, in other words it’s unorganized. Huge amounts of text data (emails, support tickets, chats, social media conversations, surveys, articles, documents, etc), is created every day but it’s hard to analyze, understand, and sort through, not to mention time-consuming and expensive.

Sentiment analysis, however, helps businesses make sense of all this unstructured text by automatically tagging it.



**Benefits of sentiment analysis**<br>
Benefits of sentiment analysis include:

1. Processes data at scale: Sentiment analysis helps businesses process huge amounts of data in an efficient and cost-effective way.

2. Real-Time analysis: Sentiment analysis can identify critical issues in real-time, for example is a PR crisis on social media escalating? Is an angry customer about to churn? Sentiment analysis models can  immediately identify, through text, these types of situations. 

3. Consistent criteria: Tagging text by sentiment is highly subjective, influenced by personal experiences, thoughts, and beliefs. It’s estimated that people only agree around 60-65% of the time when determining the sentiment of a particular text. By using a centralized sentiment analysis system, companies can apply the same criteria to all of their data, helping them improve accuracy and gain better insights.

# **Mount your Google Drive on this CoLab Notebook**
Execute the following code cell<br>
Click on the given link<br>
Select your user name<br>
Click **Allow**<br>
Copy the authorization code<br>
Paste the authorization code into the user input box. <br>
You Google Drive is mounted to this notebook. 

In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

# **Check that your drive is mounted**
1. On the menu bar, click the **folder icon**<br>
2. Click on the **folder icon with the up arrow**
3. Click on **gdrive**
4. Click on **My Drive**
5. Check for the file called **wiki-news-300d-1M.vec **<br>
If the file is there, you have correctly installed the necessary files for this notebook. <br>




# **scikit-learn**
This notebook uses the [scikit-learn](https://scikit-learn.org/stable/) library<br>
Scikit-learn is a free software machine learning library for Python.<br>
*   Simple and efficient tools for predictive data analysis
*  Accessible to everybody, and reusable in various contexts
*Built on NumPy, SciPy, and matplotlib
*Open source, commercially usable - BSD license




# **Import the libraries**

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras

In [0]:
import pandas as pd

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

# **Examine the data**<br>
The data is from three sources: <br>
> yelp reviews<br>
> amazon reviews<br>
> movie reviews<br>

The data has the structure: <br>
>"review text" label source<br>

**review text is called**: sentence<br>
**label**: 0 = negative review, 1 = positive review<br>
**source**: yelp, amazon, imdb

In [0]:
#!cat yelp_labelled.txt
#Change directory to the cloned repo
%cd /content/cloned-repo/

In [0]:
#create a dataframe containing all three sources
filepath_dict = {'yelp':   'yelp_labelled.txt',
                 'amazon': 'amazon_cells_labelled.txt',
                 'imdb':   'imdb_labelled.txt'}

df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['sentence', 'label'], sep='\t')
    df['source'] = source  # Add another column filled with the source name
    df_list.append(df)

df = pd.concat(df_list)
print(df.iloc[0])
print("dataframe shape: ",df.shape)

# **Create a Bag of Words**
The bag-of-words model is a word/sentence representation used in NLP and information retrieval. <br>
In this model, text is represented as a bag of its words, *disregarding grammar and even word order*.

Create a bag of words (BoW) for vectorizing the text. <br>
Take the data and create a vocabulary from all the words in all reviews.<br>
The collection of texts is also called a **corpus** in NLP.<br>
The **vocabulary** in this case is a list of words that occurred in our text where each word has its own index.<br>
The resulting vector is also called a **feature vector**.

In [0]:
page(2)

# **Bag of Words Example:** <br>
# **Create a bag of words (BoW) for vectorizing the text**

Create a toy set of words to demonstrate BoW.<br>

In [0]:
#1. The words
john_words = ['Sam likes to run.', 
              'John hates to be cold and he hates to run.', 
              'John hates to be late.'
            ]

In [0]:
#2. Create the bag of words
#The words are in alphabetical order (uppercase listed before lowercase)
#Each word is assigned a number
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=0, lowercase=False)
vectorizer.fit(john_words)
vocab = vectorizer.vocabulary_
vocab

In [0]:
#3. Convert the sentences in to vectors
#each word used in a sentence has a '1' in the corresponding column
dfbow = pd.DataFrame()
dfbow['sentences'] = john_words
vector = vectorizer.transform(john_words).toarray().tolist()
dfbow['vector'] = vector
print(vocab)
dfbow

Notice in the above output: 
1. All the words in all the sentences created the vocabulary.<br>
2. Each sentence is turned into a vector. <br>
3. The length of the vector is the length of the vocabulary.<br>
4. A '1' in the vector indicates the word is used in the sentence, a '0' means it is not used in the sentence. 

# **Assignment 1**: <br>
1. Write 5 sentences. 
2. Create a BoW from the sentences. 
3. Vectorize the sentences. 

Although Bag-Of-Words model is the most widely used technique for sentiment analysis, it has two major weaknesses: <br>
* the larger the vocabulary, the larger the feature vector
* it assumes each word is independent of other words
* the feature vectors are extremely sparse

A **sparse matrix**  is a matrix in which most of the elements are zero<br>
A **dense matrix**  is a matrix in which most of the elements are nonzero


# **Split the review data into train and test sets**

Split the Yelp data into training and tests sets<br>

[train_test_split](https://www.bitdegree.org/learn/train-test-split)

In [0]:
from sklearn.model_selection import train_test_split
#select the rows of the data set that are from yelp
df_yelp = df[df['source'] == 'yelp']

sentences = df_yelp['sentence'].values
y = df_yelp['label'].values

#do a 75 - 25 split between train and test data
#If int, random_state is the seed used by the random number generator; 
#If RandomState instance, random_state is the random number generator; 
#If None, the random number generator is the RandomState instance used by np.random.
sentences_train, sentences_test, y_train, y_test = train_test_split(
   sentences, y, test_size=0.25, random_state=1000)

#print out the first sentence of the training set
print(sentences_train[0])

# **Vectorize the training and test set**
Vectorize the data: <br>

Assign each word a number.<br>
Count the number of times each word appears in the individual review text. 

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
#1. use the words from the training set
#2. create a BoW from the yelp reviews
vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)
vocab = vectorizer.vocabulary_
vocab = pd.Series(vocab)
#2. vectorize the sentences
X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)
print("training data: ", X_train.shape,"\ntest data: ", X_test.shape)

What has been done so far: 
1. Created a vocabulary from all the words used in the yelp reviews.
2. Assigned each word a number.<br>

Now check the vectorization of the sentences in the yelp review training and test data. 

In [0]:
#"Select a number between 0 - 749
check=24
print(sentences_train[check])
print(X_train[check])
#Prints sentence number, word vector, quantity of word in sentence

In [0]:
#"Select a number between 0 - 749
check=24
print(sentences_test[check])
print(X_test[check])
#Prints sentence number, word vector, quantity of word in sentence

# **Assignment #2**: 
1. Split the Amazon reviews into a training and test set.
2. Create a BoW using the amazon training and test reviews. 
2. Print out the vectorization of two or three reviews from the test and training set. <br>
**Make sure your variable names for the Amazon BoW are different than the Yelp BoW.** 

# **Create a logistic regression model and train it**<br>

The [Logistic Regression class](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) from Scikit Learn implements regularized logistic regression. It can handle both dense and sparse input

In [0]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

# **Assignment 3:**

Perform logistic regression on the other two data sets<br>
>amazon<br>
imdb<br>

Get a baseline using logistic regression. This will give us something to compare with the other methods. 

In [0]:
#@title 
for source in df['source'].unique():
    df_source = df[df['source'] == source]
    reviews = df_source['sentence'].values
    reviews_y = df_source['label'].values

    reviews_train, reviews_test, reviews_y_train, reviews_y_test = train_test_split(
        reviews, reviews_y, test_size=0.25, random_state=1000)

    vectorizer = CountVectorizer()
    vectorizer.fit(reviews_train)
    r_X_train = vectorizer.transform(reviews_train)
    r_X_test  = vectorizer.transform(reviews_test)

    classifier = LogisticRegression()
    classifier.fit(r_X_train, reviews_y_train)
    score = classifier.score(r_X_test, reviews_y_test)
    print('Accuracy for {} data: {:.4f}'.format(source, score))

# **Trial 1:Keras DNN**
Create a DNN using Keras, use the Yelp reviews bag of words.<br> 
Compare it to the logistic regession using the same data. 

In [0]:
page(3)

In [0]:
input_dim = X_train.shape[1]  # Number of features
print("model imputs = ", input_dim)
model = Sequential()
model.add(layers.Dense(1700, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1000, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(100, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='relu'))

# Discussion:
Given the dataset and the model, what do you expect to happen? 

In [0]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

**Train the DNN Model**

In [0]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=20)

In [0]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

In [0]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

The Deep Neural Network trained using Bag of Words is overfit. <br>
Play with the model architecture and hyperparameters to see if you can find a better model. 

In [0]:
plot_history(history)

In the BOW model, you  represented an entire review as a single feature vector. In the next section, each word is represented as a vector. 

# **Assignment #4:**
Modify the DNN to see if you can improve the accuracy and loss. 

# **Assignment #5:** 
Train the DNN with the amazon reivews.<br>
<br>
Use different variable names than the ones used for the Yelp reviews. 

# **Word Embedding**
There are various ways to vectorize text, such as:
*   Words represented as a vector.
*   Characters represented as a vector


In this notebook, you’ll see how to deal with representing words as vectors which is the common way to use text in neural networks. Two possible ways to represent a word as a vector are:
*   one-hot encoding
*   word embeddings<br>

The first example shown below does one-hot encoding. The second example does word embeddings.



**Hot-one encoding Example**<br>
In this example you experiment with one hot encoding on a small dataset

In [0]:
#There are five cities, vocabulary size of three
food = ['bacon', 'egg', 'bacon', 'beer', 'toast']
food

Convert the foods into one their one-hot equivalents.

In [0]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
food_labels = encoder.fit_transform(food)

#create a dataframe to examine the data
df = pd.DataFrame()
df['food']= food
df['food_labels']= food_labels
df.sort_values(by=['food'])

#Convert the food into one-hot values
cat_columns = ["food_labels"]
df_processed = pd.get_dummies(df, prefix_sep="__",columns=cat_columns)
df_processed


In [0]:
#Or use the OneHotEncoder method to encode the data
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
food_labels = food_labels.reshape((5, 1))
encoder.fit_transform(food_labels)

# **Assignment #6:**
1. Create a list of 10 household appliances.
2. Convert the list into one-hot encodings

# **Discussion:** 
What are some downsides of one-hot encoding?

# **Word embedding**<br>
A word embedding is a learned representation for text where words that have the same meaning have a similar representation. It is this approach to representing words and documents that may be considered one of the key breakthroughs of deep learning on challenging natural language processing problems.




In [0]:
page(4)

Word embedding represents words as dense word vectors (also called word embeddings) which are trained unlike the one-hot encoding which are hardcoded. This means that the word embeddings collect more information into fewer dimensions.

Note that the word embeddings do not understand the text as a human would, but they rather map the statistical structure of the language used in the corpus. Their aim is to map semantic meaning into a geometric space. This geometric space is then called the embedding space.<br>

This would map semantically similar words close on the embedding space like numbers or colors. If the embedding captures the relationship between words well, things like vector arithmetic should become possible. A famous example in this field of study is the ability to map King - Man + Woman = Queen.

Word embedding has fewer dimensions than one-hot encoding<br>
Word embedding places similar words near each other<br>
One-hot encoding has a sparse matrics

There are two methods for doing word embedding: <br> 

>1.Train your word embeddings during the training of your neural network. <br>
>2.Use pretrained word embeddings which you can directly use in your model. You can leave these word embeddings unchanged during training or you can train them.<br><br>

Then tokenize the data into a format that can be used by the word embeddings. <br><br>
Keras offers a couple of convenience methods for text preprocessing and sequence preprocessing which you can employ to prepare your text.<br>

[Keras Tokenizer ](https://keras.io/preprocessing/text/)

In [0]:
from keras.preprocessing.text import Tokenizer

#Go through all the reviews and keep 3000 words.
tokenizer = Tokenizer(num_words=3000) #keep 3000 words

#Update the internal vocabulary based on a list of texts
#Must be run before running texts_to_sequences
tokenizer.fit_on_texts(sentences_train)

The number assigned to each word is dependent upon is frequency of use in all the sentences. <br>
For example:<br>
>'the' is 1<br>
'and' is 2<br>
'was' is 3<br>


In [0]:
#Examples of reviews as word embeddings
X_train = tokenizer.texts_to_sequences(sentences_train)
print(sentences_train[3],X_train[3])
print(sentences_train[23],X_train[23])
print(sentences_train[620],X_train[620])

In [0]:
X_test = tokenizer.texts_to_sequences(sentences_test)
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

print("vocab size=", vocab_size)

The indexing begins with the most common word first (the). <br>
It is important to note that the index 0 is reserved and is not assigned to any word. 

In [0]:
#Entering a word that is not in the texts will
#generate an error
for word in ['the', 'all', 'bad', 'terrible','horrible','lost','lukewarm','bacon']: 
    print('{}: {}'.format(word, tokenizer.word_index[word]))

The list can be searched by word or by index. 

In [0]:
#What is the least used word in the list? 
print((tokenizer.index_word[1746]))

# **Assignment #7:**
Use the Amazon reviews to do word embedding. 
<br>
Use different variable names than the ones used for the Yelp reviews

# **Find similar words with gensim**<br>
Gensim is an open-source library for unsupervised topic modeling and natural language processing, using modern statistical machine learning. Gensim is implemented in Python and Cython.

In [0]:
import gensim.downloader as api
word_vectors = api.load("glove-wiki-gigaword-100")
#Find words similiar to other words
#If done correctly, you can do math with words
result = word_vectors.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])
print("{}: {:.4f}".format(*result[0]))
#Try sandwich, tuna, bread

# **Assignment #8:** 
Use the gensim library to find other word equations. <br>
Share them with the class. 

# **Pad the sequence of words**

One problem that we have is that each text sequence has different number of words. To fix this, you can use pad_sequence() which simply pads the sequence of words with zeros. By default, it prepends zeros but we want to append them. Typically it does not matter whether you prepend or append zeros.

Additionally you would want to add a maxlen parameter to specify how long the sequences should be. This cuts sequences that exceed that number.

The resulting feature vector contains mostly zeros, when you have a fairly short sentence. 

In [0]:
from keras.preprocessing.sequence import pad_sequences
#The maximum length of a review, cut off the extra words 
maxlen = 100
#If a review is less than 100 words, pad the vector with 0s.

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape, "\n")

index=5
print("The review:\n",sentences_train[index])
print("\nThe final feature vector:\n",X_train[index, :])

# **Assignment #9:** 
Pad the vectors of the Amazon reviews.<br>
Use different variable names than the ones used for the Yelp reviews 

# **Train the Embedded Model**

Now you can use the Embedding Layer of Keras which takes the previously calculated integers and maps them to a dense vector of the embedding. <br>
You will need the following parameters:<br>

>input_dim: the size of the vocabulary<br>
output_dim: the size of the dense vector<br>
input_length: the length of the sequence<br>

In [0]:
page(5)

The output of the Embedding layer is a 2D vector with one embedding for each word in the input sequence of words (input document).

If you wish to connect a Dense layer directly to an Embedding layer, you must first flatten the 2D output matrix to a 1D vector using the Flatten layer.

In [0]:
embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print("input dim=",input_dim)
print("output dim of embedding layer=",embedding_dim)
print("input length = ", maxlen)
model.summary()

In [0]:
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)


# **Assignment #10**: 
Train the embedded model using the Amazon reviews

This is typically a not very reliable way to work with sequential data as you can see in the performance. When working with sequential data you want to focus on methods that look at local and sequential information instead of absolute positional information.

# **Use a MaxPooling Layer**

Another way to work with embeddings is by using a MaxPooling1D/AveragePooling1D or a GlobalMaxPooling1D/GlobalAveragePooling1D layer after the embedding. You can think of the pooling layers as a way to downsample (a way to reduce the size of) the incoming feature vectors.

In the case of max pooling you take the maximum value of all features in the pool for each feature dimension. In the case of average pooling you take the average, but max pooling seems to be more commonly used as it highlights large values.

Global max/average pooling takes the maximum/average of all features whereas in the other case you have to define the pool size. Keras has again its own layer that you can add in the sequential model:

Global max pooling = ordinary max pooling layer with pool size equals to the size of the input.<br>

Advantages of Global Pooling:
* it is more native to the convolution structure by enforcing correspondences between feature maps and categories.
* there is no parameter to optimize in the global average pooling thus overfitting is avoided at this layer. 

In [0]:
embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()


In [0]:
history = model.fit(X_train, y_train,
                    epochs=50,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)

# **Assignment #11:** 
Use the Amazon dataset to train the model with a max pooling layer. 

# **Use a precomputed embedding space**


Can performance be improved using a precomputed embedding space that utilizes a much larger corpus? <br>
It is possible to precompute word embeddings by simply training them on a large corpus of text. Among the most popular methods are Word2Vec developed by Google and GloVe (Global Vectors for Word Representation) developed by the Stanford NLP Group.<br>

Word2Vec achieves this by employing neural networks and GloVe achieves this with a co-occurrence matrix and by using matrix factorization. <br>
In both cases you are dealing with dimensionality reduction:  <br>
>Word2Vec is more accurate  <br>
GloVe is faster to compute.


In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [0]:
import numpy as np

embedding_dim = 50
embedding_matrix = create_embedding_matrix(
    '/gdrive/My Drive/wiki-news-300d-1M.vec',
    tokenizer.word_index, embedding_dim)
print(embedding_matrix.shape)

Percentage of vocabulary covered by the pretrained model

In [0]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

In [0]:
#to see the missing words: 
t = np.count_nonzero(embedding_matrix, axis=1)

count = len(t)
missing_words =[]
for i in range(1,count):
  if t[i] == 0:
    missing_words.append(tokenizer.index_word[i])

missing_words[10:20]

# **Trial 3: Embedded DNN**

In this trial use the pretrained embedded layer and the larger corpus of words. 

In [0]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=False)) #use pretrained
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

In [0]:
history = model.fit(X_train, y_train,
                    epochs=50,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)

In this trial use the larger corpus of words and train the embedded layer. 

In [0]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=True)) #train this layer
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

In [0]:
history = model.fit(X_train, y_train,
                    epochs=50,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)

# **Convolututional Neural Network (CNN)**

In [0]:
embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()


In [0]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)

You can see the 80% accuracy of the Logistic Regression model seems  tough ot beat with this data set and a CNN might not be well equipped. <br>
The reason for such a plateau might be that:
1. There are not enough training samples
2. The data you have does not generalize well
3. Missing focus on tweaking the hyperparameters

# **HyperParameter Tuning**

In [0]:
def create_model(num_filters, kernel_size, vocab_size, embedding_dim, maxlen):
    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(num_filters, kernel_size, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',metrics=["acc"])
    return model

**Embedding dimension**

In [0]:
param_grid = dict(num_filters=[32, 64, 128],
                  kernel_size=[3, 5, 7],
                  vocab_size=[5000], 
                  embedding_dim=[50],
                  maxlen=[100])

# **HyperParameter Grid Search of each text set**
Perform the random search method of hyperparameter tuning to improve the model performance. <br>


In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

# Main settings
epochs = 20
embedding_dim = 50
maxlen = 100
output_file = '/gdrive/My Drive/output.txt'

# Run grid search for each source (yelp, amazon, imdb)
for source, frame in df.groupby('source'):
    print('Running grid search for data set :', source)
    sentences = df['sentence'].values
    y = df['label'].values

    # Train-test split
    sentences_train, sentences_test, y_train, y_test = train_test_split(
        sentences, y, test_size=0.25, random_state=1000)

    # Tokenize words
    tokenizer = Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(sentences_train)
    X_train = tokenizer.texts_to_sequences(sentences_train)
    X_test = tokenizer.texts_to_sequences(sentences_test)

    # Adding 1 because of reserved 0 index
    vocab_size = len(tokenizer.word_index) + 1

    # Pad sequences with zeros
    X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
    X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

    # Parameter grid for grid search
    param_grid = dict(num_filters=[32, 64, 128],
                      kernel_size=[3, 5, 7],
                      vocab_size=[vocab_size],
                      embedding_dim=[embedding_dim],
                      maxlen=[maxlen])
    model = KerasClassifier(build_fn=create_model,
                            epochs=epochs, batch_size=10,
                            verbose=False)
    grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                              cv=4, verbose=1, n_iter=5)
    grid_result = grid.fit(X_train, y_train)

    # Evaluate testing set
    test_accuracy = grid.score(X_test, y_test)

    # Save and evaluate results
    with open(output_file, 'a') as f:
        s = ('Running {} data set\nBest Accuracy : '
             '{:.4f}\n{}\nTest Accuracy : {:.4f}\n\n')
        output_string = s.format(
            source,
            grid_result.best_score_,
            grid_result.best_params_,
            test_accuracy)
        print(output_string)
        f.write(output_string)

# **HyperParameter Tuning on all the datasets together**

In [0]:
#create a dataframe containing all three sources
filepath_dict = {'yelp':   'yelp_labelled.txt',
                 'amazon': 'amazon_cells_labelled.txt',
                 'imdb':   'imdb_labelled.txt'}

df_full_list = []
for source, filepath in filepath_dict.items():
    df_full = pd.read_csv(filepath, names=['sentence', 'label'], sep='\t')
    df_full['source'] = source  # Add another column filled with the source name
    df_full_list.append(df)

df_full = pd.concat(df_list)
print(df_full.iloc[2000])
print("dataframe shape: ",df_full.shape)

In [0]:
# Main settings
epochs = 20
embedding_dim = 50
maxlen = 100
output_file = '/gdrive/My Drive/output.txt'

print('Running grid search for data set :\n', df_full)
sentences = df['sentence'].values
y = df['label'].values

# Train-test split
sentences_train, sentences_test, y_train, y_test = train_test_split(
    sentences, y, test_size=0.25, random_state=1000)

# Tokenize words
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences_train)
X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

# Pad sequences with zeros
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Parameter grid for grid search
param_grid = dict(num_filters=[32, 64, 128],
                      kernel_size=[3, 5, 7],
                      vocab_size=[vocab_size],
                      embedding_dim=[embedding_dim],
                      maxlen=[maxlen])
model = KerasClassifier(build_fn=create_model,
                            epochs=epochs, batch_size=10,
                            verbose=False)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                              cv=4, verbose=1, n_iter=5)
grid_result = grid.fit(X_train, y_train)

# Evaluate testing set
test_accuracy = grid.score(X_test, y_test)

# Save and evaluate results
with open(output_file, 'a') as f:
   s = ('Running {} data set\nBest Accuracy : '
        '{:.4f}\n{}\nTest Accuracy : {:.4f}\n\n')
   output_string = s.format(
            source,
            grid_result.best_score_,
            grid_result.best_params_,
            test_accuracy)
   print(output_string)
   f.write(output_string)